# Introduction

A recent study confirmed the association between gut microbiota and shifts in host healthy status: The gut microbiota is involved in colorectal cancer (CRC) progression [REF]. Therewith, it holds great potential to investigate whether the progression of CRC could be monitored through the gut microbiota. We assessed such applicability of EXPERT by introducing cancer data from Zeller, G. et al. In this assessment, we considered five stages in the progression of colorectal cancer (CRC): 0 (Healthy control) I, II, III, and IV according to the study of Zeller G. et al. We first found that the compositional shifts of the human gut within such progression are invisible to some traditional methods, exemplified by Principle Coordination Analysis (PCoA) using distance metric either in weighted-Unifrac or Jensen Shannon divergence. However, the assessment result of utilizing cross-validation accord with our hypothesis: the progression stage of CRC can be accurately monitored, while an obviously better performance (ROC-AUC over 0.95 for stages from I to IV) was achieved by the model built from the Disease Model. This has proved the superior applicability of EXPERT as a method for early detection of the occurrence of cancers and suggested considerable potential optimization through knowledge transfer in such monitor systems.

# Reproducibility statement

- EXPERT supports completely reproducible optimization & inference.
- Processed data are provided for reproducing the result, the original data can be found under `dataFiles/`.
- Rerunning the entire notebook with the configuration below should yield **completely consistent** results (compared to those reported in our paper).
- Session information
    - EXPERT (version 0.3)
    - Python (version 3.8.2)
    - TensorFlow (version 2.3.1)
    - Pandas (version 1.1.3)
    - NumPy (version 1.18.5)
    - ETE3 (version 3.1.2)
    - NCBI taxonomy database (released [2020-09-01](https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump_archive/))

## Process
The following sections are used to reproduce the result reported in our paper. For detailed configuration and interpretation of results, please read our original paper first.

### Optimization
- `--finetune`: enable finetune for further optimization.
- `--update-statistics`: update statistics for Z-score standardization.

In [1]:
%%bash
for i in {0,1,2,3,4}; do
    expert train -i experiments/exp_$i/SourceCM.h5 -t ontology.pkl -l experiments/exp_$i/SourceLabels.h5 -o experiments/exp_$i/Independent;
    expert transfer -i experiments/exp_$i/SourceCM.h5 -t ontology.pkl \
        -l experiments/exp_$i/SourceLabels.h5 -o experiments/exp_$i/Transfer_HM \
        -m HumanModel/ --finetune --update-statistics;
    expert transfer -i experiments/exp_$i/SourceCM.h5 -t ontology.pkl \
        -l experiments/exp_$i/SourceLabels.h5 -o experiments/exp_$i/Transfer_DM \
        -m ../Disease-diagnosis/experiments/exp_3/TrainModel/ --finetune --update-statistics;
done

Process is terminated.


### Quantifying source contributions

- `--measure-unknown`: measure the contribution from unknown source(s).

In [32]:
%%bash
for i in {0,1,2,3,4}; do
    expert search -i experiments/exp_$i/QueryCM.h5 -m experiments/exp_$i/Independent -o experiments/exp_$i/Search_Independent;
    expert search -i experiments/exp_$i/QueryCM.h5 -m experiments/exp_$i/Transfer_HM -o experiments/exp_$i/Search_Transfer_HM;
    expert search -i experiments/exp_$i/QueryCM.h5 -m experiments/exp_$i/Transfer_DM -o experiments/exp_$i/Search_Transfer_DM;
done

Reordering labels and samples...
Total matched samples: 571
N. NaN in input features: 0
           mean       std
0      0.000000  0.000000
1      0.000000  0.000000
2      0.000000  0.000000
3      0.015062  0.041284
4      0.015041  0.041268
...         ...       ...
18013  0.000061  0.000243
18014  0.000000  0.000000
18015  0.002382  0.011947
18016  0.002317  0.011715
18017  0.000000  0.000000

[18018 rows x 2 columns]
Pre-training using Adam with lr=1e-05...
Epoch 1/2
1/1 [==============================] - 0s 411ms/step - loss: 0.7021 - acc: 0.4877 - val_loss: 0.7029 - val_acc: 0.5069
Epoch 2/2
1/1 [==============================] - 0s 22ms/step - loss: 0.6914 - acc: 0.5099 - val_loss: 0.6967 - val_acc: 0.5448
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 18018)]           0         
________________________________________________________

mkdir: cannot create directory ‘running_2’: File exists
2020-11-26 13:23:18.622388: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-26 13:23:18.629832: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2499960000 Hz
2020-11-26 13:23:18.631653: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x563da94111d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-26 13:23:18.631681: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-11-26 13:23:19.640297: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 144288144 exceeds 10% of free system memory.
2020-11-26 13:23:20.7306

### Evaluating performances
- `-S`: Set threshold for evaluation

In [ ]:
%%bash
for i in {0,1,2,3,4}; do
    expert evaluate -i experiments/exp_$i/Search_Independent -l experiments/exp_$i/QueryLabels.h5 -o experiments/exp_$i/Eval_Independent -S 0;
    expert evaluate -i experiments/exp_$i/Search_Transfer_HM -l experiments/exp_$i/QueryLabels.h5 -o experiments/exp_$i/Eval_Transfer_HM -S 0;
    expert evaluate -i experiments/exp_$i/Search_Transfer_DM -l experiments/exp_$i/QueryLabels.h5 -o experiments/exp_$i/Eval_Transfer_DM -S 0;
done

## Support
For support reproducing the result, please email: huichong.me@gmail.com.